<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/%EC%8B%9C%EA%B3%84%EC%97%B4_1%ED%8C%80_0612_private_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test 데이터 전처리 후 inference
=> ulsan_weather_06, dangjin_weather_06 최신 업데이트 버전으로 가져오기

* 이미지 한 장이 하루여야 함
* 72시간의 데이터를 갖고 73번째 시간대의 데이터를 예측하는것

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at ./gdrive


In [ ]:
'''
dangjin_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_dangjin_fcst.csv")
ulsan_fcst = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_ulsan_fcst.csv")
energy = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/new_energy.csv")
'''

In [ ]:
dangjin_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/dangjin_weather_06.csv")
ulsan_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/ulsan_weather_06.csv")

In [ ]:
dangjin_weather_06

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed,방위각,고도,적경,적위
0,2021-06-01 01:00:00,76.0,7.0,15.9,90.0,0.5,007 36 14.13,-30 32 07.22,04 36 14.12,22 02 21.29
1,2021-06-01 02:00:00,76.0,5.0,15.1,0.0,0.3,023 04 32.21,-27 21 03.06,04 36 24.48,22 02 41.83
2,2021-06-01 03:00:00,77.0,6.0,15.7,70.0,0.7,036 50 58.64,-21 21 08.54,04 36 34.83,22 03 02.46
3,2021-06-01 04:00:00,72.0,7.0,16.5,0.0,0.2,048 39 27.91,-13 12 43.90,04 36 45.16,22 03 23.15
4,2021-06-01 05:00:00,85.0,6.0,15.1,0.0,0.0,058 46 13.21,-3 33 24.30,04 36 55.46,22 03 43.89
...,...,...,...,...,...,...,...,...,...,...
307,2021-06-13 19:00:00,60.0,1.0,26.0,312.0,4.0,292 27 31.54,08 55 09.01,05 28 50.73,23 14 31.66
308,2021-06-13 20:00:00,60.0,1.0,26.0,312.0,4.0,301 11 27.42,-1 45 28.52,05 29 01.15,23 14 38.84
309,2021-06-13 21:00:00,60.0,1.0,26.0,312.0,4.0,311 04 46.48,-11 25 32.70,05 29 11.59,23 14 46.04
310,2021-06-13 22:00:00,60.0,1.0,26.0,312.0,4.0,322 34 53.61,-19 37 19.98,05 29 22.07,23 14 53.28


In [ ]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
  #dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
  #dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  #dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
  #dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
  return dangjin_fcst

In [ ]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [ ]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))


In [ ]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  #Q. 어차피 다 나눠놨는데 싸인 코싸인은 왜 씌우는것..?
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']


dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

#dangjin_weather_06['WindDirection_catg'] = dangjin_weather_06['WindDirection'].apply(angle_to_dir)
#ulsan_weather_06['WindDirection_catg'] = ulsan_weather_06['WindDirection'].apply(angle_to_dir)

In [ ]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [ ]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [ ]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [ ]:
#형태 다듬기 (?)
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [ ]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각', '고도', '적경', '적위']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [ ]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,15.9,11.672001,76.0,7.0,5.000000e-01,6.123234e-17,1,6,7,-30,4.3614,22.0221
1,15.1,10.897865,76.0,5.0,1.836970e-17,3.000000e-01,2,6,23,-27,4.3624,22.0241
2,15.7,11.676429,77.0,6.0,6.577848e-01,2.394141e-01,3,6,36,-21,4.3634,22.0302
3,16.5,11.432089,72.0,7.0,1.224647e-17,2.000000e-01,4,6,48,-13,4.3645,22.0323
4,15.1,12.594910,85.0,6.0,0.000000e+00,0.000000e+00,5,6,58,-3,4.3655,22.0343
...,...,...,...,...,...,...,...,...,...,...,...,...
307,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,19,6,292,8,5.2850,23.1431
308,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,20,6,301,-1,5.2901,23.1438
309,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,21,6,311,-11,5.2911,23.1446
310,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,22,6,322,-19,5.2922,23.1453


In [ ]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71)     

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,...,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.9,11.672001,76.0,7.0,5.000000e-01,6.123234e-17,1,6,7,-30,4.3614,22.0221
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.9,11.672001,76.0,7.0,5.000000e-01,6.123234e-17,1.0,6.0,7.0,-30.0,4.3614,22.0221,15.1,10.897865,76.0,5.0,1.836970e-17,3.000000e-01,2,6,23,-27,4.3624,22.0241
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.9,11.672001,76.0,7.0,5.000000e-01,6.123234e-17,1.0,6.0,7.0,-30.0,4.3614,22.0221,15.1,10.897865,76.0,5.0,1.836970e-17,3.000000e-01,2.0,6.0,23.0,-27.0,4.3624,22.0241,15.7,11.676429,77.0,6.0,6.577848e-01,2.394141e-01,3,6,36,-21,4.3634,22.0302
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,-30.0,4.3614,22.0221,15.1,10.897865,76.0,5.0,1.836970e-17,3.000000e-01,2.0,6.0,23.0,-27.0,4.3624,22.0241,15.7,11.676429,77.0,6.0,6.577848e-01,2.394141e-01,3.0,6.0,36.0,-21.0,4.3634,22.0302,16.5,11.432089,72.0,7.0,1.224647e-17,2.000000e-01,4,6,48,-13,4.3645,22.0323
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,-27.0,4.3624,22.0241,15.7,11.676429,77.0,6.0,6.577848e-01,2.394141e-01,3.0,6.0,36.0,-21.0,4.3634,22.0302,16.5,11.432089,72.0,7.0,1.224647e-17,2.000000e-01,4.0,6.0,48.0,-13.0,4.3645,22.0323,15.1,12.594910,85.0,6.0,0.000000e+00,0.000000e+00,5,6,58,-3,4.3655,22.0343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,23.5,16.538120,65.0,10.0,1.607495e+00,-4.416555,21.0,6.0,311.0,-11.0,5.1644,23.0345,20.8,18.737684,88.0,10.0,1.162868e+00,-3.194955,22.0,6.0,322.0,-19.0,5.1655,23.0355,20.4,18.161646,87.0,10.0,3.735173e-16,-6.100000,23.0,6.0,336.0,-25.0,5.1705,23.0406,20.7,19.176528,91.0,10.0,...,267.0,43.0,5.2819,23.1409,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,17.0,6.0,276.0,32.0,5.2830,23.1417,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,18.0,6.0,284.0,20.0,5.2840,23.1424,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,19,6,292,8,5.2850,23.1431
308,20.8,18.737684,88.0,10.0,1.162868e+00,-3.194955,22.0,6.0,322.0,-19.0,5.1655,23.0355,20.4,18.161646,87.0,10.0,3.735173e-16,-6.100000,23.0,6.0,336.0,-25.0,5.1705,23.0406,20.7,19.176528,91.0,10.0,3.673940e-16,-6.000000,0.0,6.0,351.0,-29.0,5.1716,23.0416,20.7,19.698490,94.0,10.0,...,276.0,32.0,5.2830,23.1417,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,18.0,6.0,284.0,20.0,5.2840,23.1424,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,19.0,6.0,292.0,8.0,5.2850,23.1431,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522e+00,20,6,301,-1,5.2901,23.1438
309,20.4,18.161646

In [ ]:
'''
dangjin_fcst_train = dangjin_fcst.iloc[:-24 * 28]
dangjin_fcst_test = dangjin_fcst.iloc[-24 * 28:]

ulsan_fcst_train = ulsan_fcst.iloc[:-24 * 28]
ulsan_fcst_test = ulsan_fcst.iloc[-24 * 28:]
'''

In [ ]:
#앞에 72개는 nan 제거

dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]

In [ ]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,...,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위
71,15.9,11.672001,76.0,7.0,5.000000e-01,6.123234e-17,1.0,6.0,7.0,-30.0,4.3614,22.0221,15.1,10.897865,76.0,5.0,1.836970e-17,0.300000,2.0,6.0,23.0,-27.0,4.3624,22.0241,15.7,11.676429,77.0,6.0,6.577848e-01,0.239414,3.0,6.0,36.0,-21.0,4.3634,22.0302,16.5,11.432089,72.0,7.0,...,310.0,-12.0,4.4751,22.2358,14.5,13.380556,93.0,10.0,1.744303,4.792432,22.0,6.0,322.0,-20.0,4.4801,22.2415,13.9,12.620588,92.0,10.0,2.537170e+00,0.923454,23.0,6.0,336.0,-26.0,4.4811,22.2433,14.2,13.247144,94.0,10.0,1.503508e+00,0.547232,0,6,351,-30,4.4822,22.2450
72,15.1,10.897865,76.0,5.0,1.836970e-17,3.000000e-01,2.0,6.0,23.0,-27.0,4.3624,22.0241,15.7,11.676429,77.0,6.0,6.577848e-01,0.239414,3.0,6.0,36.0,-21.0,4.3634,22.0302,16.5,11.432089,72.0,7.0,1.224647e-17,0.200000,4.0,6.0,48.0,-13.0,4.3645,22.0323,15.1,12.594910,85.0,6.0,...,322.0,-20.0,4.4801,22.2415,13.9,12.620588,92.0,10.0,2.537170,0.923454,23.0,6.0,336.0,-26.0,4.4811,22.2433,14.2,13.247144,94.0,10.0,1.503508e+00,0.547232,0.0,6.0,351.0,-30.0,4.4822,22.2450,14.2,12.917607,92.0,9.0,1.710101e-01,0.469846,1,6,7,-30,4.4832,22.2508
73,15.7,11.676429,77.0,6.0,6.577848e-01,2.394141e-01,3.0,6.0,36.0,-21.0,4.3634,22.0302,16.5,11.432089,72.0,7.0,1.224647e-17,0.200000,4.0,6.0,48.0,-13.0,4.3645,22.0323,15.1,12.594910,85.0,6.0,0.000000e+00,0.000000,5.0,6.0,58.0,-3.0,4.3655,22.0343,17.5,10.134558,62.0,9.0,...,336.0,-26.0,4.4811,22.2433,14.2,13.247144,94.0,10.0,1.503508,0.547232,0.0,6.0,351.0,-30.0,4.4822,22.2450,14.2,12.917607,92.0,9.0,1.710101e-01,0.469846,1.0,6.0,7.0,-30.0,4.4832,22.2508,13.2,12.733699,97.0,7.0,0.000000e+00,0.000000,2,6,22,-27,4.4842,22.2525
74,16.5,11.432089,72.0,7.0,1.224647e-17,2.000000e-01,4.0,6.0,48.0,-13.0,4.3645,22.0323,15.1,12.594910,85.0,6.0,0.000000e+00,0.000000,5.0,6.0,58.0,-3.0,4.3655,22.0343,17.5,10.134558,62.0,9.0,-6.429396e-16,3.500000,6.0,6.0,67.0,7.0,4.3705,22.0404,18.4,11.930196,66.0,8.0,...,351.0,-30.0,4.4822,22.2450,14.2,12.917607,92.0,9.0,0.171010,0.469846,1.0,6.0,7.0,-30.0,4.4832,22.2508,13.2,12.733699,97.0,7.0,0.000000e+00,0.000000,2.0,6.0,22.0,-27.0,4.4842,22.2525,13.2,12.890527,98.0,7.0,-1.102182e-16,0.600000,3,6,36,-21,4.4853,22.2543
75,15.1,12.594910,85.0,6.0,0.000000e+00,0.000000e+00,5.0,6.0,58.0,-3.0,4.3655,22.0343,17.5,10.134558,62.0,9.0,-6.429396e-16,3.500000,6.0,6.0,67.0,7.0,4.3705,22.0404,18.4,11.930196,66.0,8.0,1.455484e+00,1.221296,7.0,6.0,75.0,18.0,4.3715,22.0425,18.9,12.635153,67.0,8.0,...,7.0,-30.0,4.4832,22.2508,13.2,12.733699,97.0,7.0,0.000000,0.000000,2.0,6.0,22.0,-27.0,4.4842,22.2525,13.2,12.890527,98.0,7.0,-1.102182e-16,0.600000,3.0,6.0,36.0,-21.0,4.4853,22.2543,12.7,12.391732,98.0,7.0,4.898587e-17,-0.800000,4,6,48,-12,4.4903,22.2601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,23.5,16.538120,65.0,10.0,1.607495e+00,-4.416555e+00,21.0,6.0,311.0,-11.0,5.1644,23.0345,20.8,18.737684,88.0,10.0,1.162868e+00,-3.194955,22.0,6.0,322.0,-19.0,5.1655,23.0355,20.4,18.161646,87.0,10.0,3.735173e-16,-6.100000,23.0,6.0,336.0,-25.0,5.1705,23.0406,20.7,19.176528,91.0,10.0,...,267.0,43.0,5.2819,23.1409,26.0,17.632037,60.0,1.0,-2.972579,2.676522,17.0,6.0,276.0,32.0,5.2830,23.1417,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676522,18.0,6.0,284.0,20.0,5.2840,23.1424,26.0,17.632037,60.0,1.0,-2.972579e+00,2.676

In [ ]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [ ]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [ ]:
import tensorflow as tf

reshape 해줌 => 이미지 가로 x 세로 해서 1296만 나오면 됨

하루에 해당하는 데이터가 1296이라서.ㅇㅇ

1296은 (72일(shift) 데이터) x (column 18개) 이다.

In [ ]:
dangjin_weather_06.shape

(241, 864)

In [ ]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 12, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 12, 1))

In [ ]:
dangjin_weather_06.shape

(241, 72, 12, 1)

In [ ]:
'''
dangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) 
dangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))

ulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))
ulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))
'''

'\ndangjin_fcst_train = tf.image.resize(dangjin_fcst_train, (72, 32)) \ndangjin_fcst_test = tf.image.resize(dangjin_fcst_test, (72, 32))\n\nulsan_fcst_train = tf.image.resize(ulsan_fcst_train, (72, 32))\nulsan_fcst_test = tf.image.resize(ulsan_fcst_test, (72, 32))\n'

In [ ]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [ ]:
ulsan_weather_06.shape

(241, 72, 12, 3)

In [ ]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

In [ ]:
dangjin_test.shape

(24, 72, 12, 3)

* Inference

In [ ]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [ ]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [ ]:
floating3

array([[  4.1642866],
       [  4.1319904],
       [  4.652999 ],
       [  5.3104715],
       [  5.8972178],
       [  7.677016 ],
       [ 11.7698765],
       [ 21.898176 ],
       [ 34.521683 ],
       [ 63.667835 ],
       [160.07568  ],
       [279.0697   ],
       [393.27612  ],
       [479.4853   ],
       [488.52475  ],
       [443.07974  ],
       [316.0294   ],
       [181.72922  ],
       [ 80.3556   ],
       [ 22.96119  ],
       [ 10.835392 ],
       [  6.04546  ],
       [  4.4860997],
       [  4.3186045]], dtype=float32)

In [ ]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [ ]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [ ]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [ ]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [ ]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [ ]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [ ]:
floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5

In [ ]:
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")

In [ ]:
#sample_submission.loc[743:766] #6/12일
sample_submission.loc[767:790] #6/13일    #loc는 끝 인덱스도 포함....

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
767,2021-06-12 24:00:00,0,0,0,0
768,2021-06-13 01:00:00,0,0,0,0
769,2021-06-13 02:00:00,0,0,0,0
770,2021-06-13 03:00:00,0,0,0,0
771,2021-06-13 04:00:00,0,0,0,0
772,2021-06-13 05:00:00,0,0,0,0
773,2021-06-13 06:00:00,0,0,0,0
774,2021-06-13 07:00:00,0,0,0,0
775,2021-06-13 08:00:00,0,0,0,0
776,2021-06-13 09:00:00,0,0,0,0


In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[767:790].shape

(24, 5)

In [ ]:
sample_submission.loc[767:790, 'dangjin_floating'] = floating
sample_submission.loc[767:790, 'dangjin_warehouse'] = warehouse
sample_submission.loc[767:790, 'dangjin'] = dangjin
sample_submission.loc[767:790, 'ulsan'] = ulsan

In [ ]:
sample_submission.loc[766:791]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
766,2021-06-12 23:00:00,0.000000,0.000000,0.000000,0.000000
767,2021-06-12 24:00:00,5.992369,5.223150,13.645849,15.432528
768,2021-06-13 01:00:00,7.077891,8.561481,12.768064,24.636524
769,2021-06-13 02:00:00,7.181551,11.918996,12.866567,32.891727
770,2021-06-13 03:00:00,6.684646,16.142807,14.113556,39.352192
771,2021-06-13 04:00:00,8.285769,21.090893,16.095493,44.785057
772,2021-06-13 05:00:00,9.254688,23.120861,20.577190,51.454926
773,2021-06-13 06:00:00,9.635546,24.051975,18.769871,61.477959
774,2021-06-13 07:00:00,13.999456,32.088367,24.614481,86.428764
775,2021-06-13 08:00:00,26.907581,45.867943,44.564167,131.539368


In [ ]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0613_submission.csv", index = False)

In [ ]:
ls

0608_cnn_5fold.csv  gdrive/  sample_data/
